# Inserting Multiple Documents

In [1]:
import pymongo
client = pymongo.MongoClient()
db = client.test

In [2]:
videos = [
    {'title': 'Python Design Patterns'},
    {'title': 'Bootstrap 4 Projects'},
    {'title': 'Ethical Hacking for Beginners'},
    {'title': 'Expert Ruby on Rails 5'}
]
db.packt_videos.drop()
result = db.packt_videos.insert_many(videos)
result

In [3]:
result.inserted_ids

[ObjectId('5b3418ae722c7d6718c731c2'),
 ObjectId('5b3418ae722c7d6718c731c3'),
 ObjectId('5b3418ae722c7d6718c731c4'),
 ObjectId('5b3418ae722c7d6718c731c5')]

In [4]:
list(db.packt_videos.find())

[{'_id': ObjectId('5b3418ae722c7d6718c731c2'),
  'title': 'Python Design Patterns'},
 {'_id': ObjectId('5b3418ae722c7d6718c731c3'),
  'title': 'Bootstrap 4 Projects'},
 {'_id': ObjectId('5b3418ae722c7d6718c731c4'),
  'title': 'Ethical Hacking for Beginners'},
 {'_id': ObjectId('5b3418ae722c7d6718c731c5'),
  'title': 'Expert Ruby on Rails 5'}]

# Compound Bulk Operations

In [5]:
from pprint import pprint
from pymongo import UpdateMany


videos = [
    {'title': 'Python Design Patterns', 'authors': ['Gergo Bogdan']},
    {'title': 'Bootstrap 4 Projects', 'authors': ['Aaron Vanston']},
    {'title': 'Ethical Hacking for Beginners', 'authors': ['Fred Heath']},
    {'title': 'OpenFrame works Interactivity', 'authors': ['Denis Perevalov', 'Igor Tatarnikov']}
]

bulk = db.packt_videos.bulk_write([
    UpdateMany({'title': video['title']}, {'$addToSet': {'authors': {'$each': video['authors']}}}, upsert=True)
for video in videos])

pprint(bulk.bulk_api_result)

{'nInserted': 0,
 'nMatched': 3,
 'nModified': 3,
 'nRemoved': 0,
 'nUpserted': 1,
 'upserted': [{'_id': ObjectId('5b3418aecd139b5227d8ad69'), 'index': 3}],
 'writeConcernErrors': [],
 'writeErrors': []}


In [6]:
list(db.packt_videos.find({}, {'_id': 0}))

[{'title': 'Python Design Patterns', 'authors': ['Gergo Bogdan']},
 {'title': 'Bootstrap 4 Projects', 'authors': ['Aaron Vanston']},
 {'title': 'Ethical Hacking for Beginners', 'authors': ['Fred Heath']},
 {'title': 'Expert Ruby on Rails 5'},
 {'title': 'OpenFrame works Interactivity',
  'authors': ['Denis Perevalov', 'Igor Tatarnikov']}]

# Handling Errors

In [7]:
doc = db.packt_videos.find_one()
doc

{'_id': ObjectId('5b3418ae722c7d6718c731c2'),
 'title': 'Python Design Patterns',
 'authors': ['Gergo Bogdan']}

In [8]:
from pymongo import InsertOne


try:
    db.packt_videos.bulk_write([
        InsertOne(doc)
    ])
    print('success')
except pymongo.errors.BulkWriteError as bwe:
    pprint(bwe.details)

{'nInserted': 0,
 'nMatched': 0,
 'nModified': 0,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': [{'code': 11000,
                  'errmsg': 'E11000 duplicate key error collection: '
                            'test.packt_videos index: _id_ dup key: { : '
                            "ObjectId('5b3418ae722c7d6718c731c2') }",
                  'index': 0,
                  'op': {'_id': ObjectId('5b3418ae722c7d6718c731c2'),
                         'authors': ['Gergo Bogdan'],
                         'title': 'Python Design Patterns'}}]}


In [9]:
bulk = db.packt_videos.bulk_write([
    UpdateMany({'title': video['title']}, {'$addToSet': {'authors': {'$each': video['authors']}}}, upsert=True)
for video in videos], ordered=False)

pprint(bulk.bulk_api_result)

{'nInserted': 0,
 'nMatched': 4,
 'nModified': 0,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': []}
